In [1]:
# For plotting
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
# For conversion
from skimage.color import lab2rgb, rgb2lab, rgb2gray
from skimage import io
# For everything
import torch
import torch.nn as nn
import torch.nn.functional as F
# For our model
import torchvision
import torchvision.models as models
from torchvision import datasets, transforms
from torchmetrics import MeanSquaredError, PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
from PIL import Image
# For utilities
import os, shutil, time

In [2]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [3]:
# colab i kaggle jeszcze nie testowane
colab = False
kaggle = False
test_number = '12_2'

In [4]:
color_imgs = 'outputs/color/'
gray_imgs = 'outputs/gray/'
checkpoints = 'checkpoints'
if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    dataset = '/content/drive/MyDrive/MGU/cifar10/'
    
    color_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{color_imgs}'
    gray_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{gray_imgs}'
    checkpoints = f'/content/drive/MyDrive/MGU/{test_number}/{checkpoints}'
elif kaggle:
    dataset = '/kaggle/input/cifar10/'
    
    color_imgs = f'{test_number}/{color_imgs}'
    gray_imgs = f'{test_number}/{gray_imgs}'
    checkpoints = f'{test_number}/{checkpoints}'
else:
    dataset = '../../datasets/cifar10/'

In [5]:
# Make folders and set parameters
os.makedirs(color_imgs, exist_ok=True)
os.makedirs(gray_imgs, exist_ok=True)
os.makedirs(checkpoints, exist_ok=True)
save_images = True
best_losses = [1e10, 1e10, 1e10]
best_epoch = -1
patience = 50
epochs = 500
batch_size = 128
SIZE = 32

In [6]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [7]:
# Check if GPU is available
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [8]:
class LabImageFolder(torch.utils.data.Dataset):
    def __init__(self, paths, split='train'):
        if split == 'train':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC),
                transforms.RandomCrop(SIZE),
                transforms.RandomHorizontalFlip(),  
                transforms.ToTensor(),
                transforms.Normalize((0.4918019, 0.48263696, 0.44733968), (0.24705184, 0.2433789, 0.26162848)),
                transforms.ToPILImage(),
            ])
        elif split == 'val':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC), 
                transforms.RandomCrop(SIZE), 
                transforms.ToTensor(),
                transforms.Normalize((0.4918019, 0.48263696, 0.44733968), (0.24705184, 0.2433789, 0.26162848)),
                transforms.ToPILImage(),
            ])
            
        self.split = split
        self.size = SIZE
        self.paths = [os.path.join(paths, file) for file in os.listdir(paths) if os.path.isfile(
            os.path.join(paths, file))]
        
        
    def __getitem__(self, index):
        img = Image.open(self.paths[index]).convert("RGB")
        img_original = self.transforms(img)
        img_original = np.asarray(img_original)
        img_lab = rgb2lab(img_original)
        img_lab = (img_lab + 128) / 255
        img_ab = img_lab[:, :, 1:3]
        img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
        img_gray = rgb2gray(img_original)
        img_gray = torch.from_numpy(img_gray).unsqueeze(0).float()
        return img_gray, img_ab
    
    def __len__(self):
        return len(self.paths)

In [9]:
# Training
train_imagefolder = LabImageFolder(dataset + 'train')
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=batch_size, shuffle=True)
# Validation 
val_imagefolder = LabImageFolder(dataset + 'val' , 'val')
val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=batch_size, shuffle=False)

In [10]:
kernel_size=3
stride_en=2
stride_de=1
padding=1
scale_factor=2
padding_mode='zeros'
channels_base = 16
p1 = .25
p2 = .5

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        self.conv1 = nn.Conv2d(1, channels_base, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)
        self.conv2 = nn.Conv2d(channels_base, channels_base * 2, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)
        
        self.convtrans1 = nn.ConvTranspose2d(channels_base * 2, channels_base, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans2 = nn.ConvTranspose2d(channels_base, channels_base // 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans3 = nn.ConvTranspose2d(channels_base // 2, 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)

        self.batchnorm1 = nn.BatchNorm2d(channels_base // 2)
        self.batchnorm2 = nn.BatchNorm2d(channels_base)
        self.batchnorm3 = nn.BatchNorm2d(channels_base * 2)
        
        self.dropout1 = nn.Dropout(p=p1)
        self.dropout2 = nn.Dropout(p=p2)
        
        
    def forward(self, input):
        # encoder
        x = F.relu(self.batchnorm2(self.conv1(input)))
        x = self.dropout1(x)
        x = F.relu(self.batchnorm3(self.conv2(x)))
        x = self.dropout2(x)
        
        # decoder
        x = F.relu(self.batchnorm2(self.convtrans1(x)))
        x = self.dropout2(x)
        x = F.interpolate(x, scale_factor=scale_factor)
        x = F.relu(self.batchnorm1(self.convtrans2(x)))
        x = self.dropout1(x)
        x = F.interpolate(self.convtrans3(x), scale_factor=scale_factor)

        return x

In [11]:
model = Autoencoder()

In [12]:
criterion = [MeanSquaredError(), PeakSignalNoiseRatio(data_range=1.0), StructuralSimilarityIndexMeasure(data_range=1.0)]

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [14]:
# # Move model and loss function to GPU
if use_gpu: 
    criterion = [criterion[0].to("cuda"), criterion[1].to("cuda"), criterion[2].to("cuda")]
    model = model.cuda()

In [15]:
if use_gpu: 
    from torchsummary import summary
    summary(model, (1, SIZE, SIZE))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 16, 16]             160
       BatchNorm2d-2           [-1, 16, 16, 16]              32
           Dropout-3           [-1, 16, 16, 16]               0
            Conv2d-4             [-1, 32, 8, 8]           4,640
       BatchNorm2d-5             [-1, 32, 8, 8]              64
           Dropout-6             [-1, 32, 8, 8]               0
   ConvTranspose2d-7             [-1, 16, 8, 8]           4,624
       BatchNorm2d-8             [-1, 16, 8, 8]              32
           Dropout-9             [-1, 16, 8, 8]               0
  ConvTranspose2d-10            [-1, 8, 16, 16]           1,160
      BatchNorm2d-11            [-1, 8, 16, 16]              16
          Dropout-12            [-1, 8, 16, 16]               0
  ConvTranspose2d-13            [-1, 2, 16, 16]             146
Total params: 10,874
Trainable params: 

In [16]:
class AverageMeter(object):
    '''A handy class from the PyTorch ImageNet tutorial''' 
    def __init__(self):
        self.reset()
    def reset(self):
        self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def to_rgb(grayscale_input, ab_input, save_path=None, save_name=None):
    '''Show/save rgb image from grayscale and ab channels
       Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
    plt.clf() # clear matplotlib 
    color_image = torch.cat((grayscale_input, ab_input), 0).numpy() # combine channels
    color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
    color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
    color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
    color_image = lab2rgb(color_image.astype(np.float64))
    grayscale_input = grayscale_input.squeeze().numpy()
    if save_path is not None and save_name is not None: 
        plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path['grayscale'], save_name), cmap='gray')
        plt.imsave(arr=color_image, fname='{}{}'.format(save_path['colorized'], save_name))

In [17]:
def validate(val_loader, model, criterion, save_images, epoch):
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]

    model.eval()
    already_saved_images = False
    for gray, ab in val_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()

        # Run model and record loss
        output_ab = model(gray) # throw away class predictions
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))

        # Save images to file
        if save_images and not already_saved_images:
            already_saved_images = True
            for j in range(min(len(output_ab), 10)): # save at most 5 images
                save_path = {'grayscale': gray_imgs, 'colorized': color_imgs}
                save_name = 'img-{}-epoch-{}.jpg'.format(j, epoch)
                to_rgb(gray[j].cpu(), ab_input=output_ab[j].detach().cpu(), save_path=save_path, save_name=save_name)

    print(f'Validate: MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print('Finished validation.')
    if epoch >= 0:
        writer.add_scalar("MSE/test", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/test", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/test", _loss[2].avg, epoch)
    return _loss[0].avg, _loss[1].avg, _loss[2].avg

In [18]:
def train(train_loader, model, criterion, optimizer, epoch):
    print(f'Starting training epoch {epoch}')
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]
    
    model.train()

    for gray, ab in train_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()
            
        optimizer.zero_grad()

        output_ab = model(gray) 
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        loss[0].backward()
        optimizer.step()
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))
        
    print(f'Epoch: {epoch}, MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print(f'Finished training epoch {epoch}')
    if epoch >= 0:
        writer.add_scalar("MSE/train", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/train", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/train", _loss[2].avg, epoch)

In [19]:
# Train model
for epoch in range(epochs):
    # Train for one epoch, then validate
    train(train_loader, model, criterion, optimizer, epoch)
    with torch.no_grad():
        losses = validate(val_loader, model, criterion, save_images, epoch)
    # Save checkpoint and replace old best model if current model is better
    if losses[0] < best_losses[0]:
        best_losses[0] = losses[0]
        best_epoch = epoch
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}.pth')
    if losses[1] < best_losses[1]:
        best_losses[1] = losses[1]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-PSNRLoss-{losses[1]:.8f}.pth')
    if losses[2] < best_losses[2]:
        best_losses[2] = losses[2]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-SSIMLoss-{losses[2]:.8f}.pth')
    
    if epoch - best_epoch >= patience:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}-early_stop.pth')
        break
    
    if epoch == epochs - 1:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')


Starting training epoch 0
Epoch: 0, MSE 0.02550463 (0.03291292), PSNR 15.93380928 (15.50281539), SSIM 0.09644152 (0.08457108)
Finished training epoch 0
Validate: MSE 0.02373184 (0.02537501), PSNR 16.24668694 (15.96229132), SSIM 0.06977409 (0.09456435)
Finished validation.
Starting training epoch 1
Epoch: 1, MSE 0.02357968 (0.02399403), PSNR 16.27462006 (16.20026162), SSIM 0.11629625 (0.10420734)
Finished training epoch 1
Validate: MSE 0.02335386 (0.02499233), PSNR 16.31641197 (16.02830475), SSIM 0.06976228 (0.09474481)
Finished validation.
Starting training epoch 2
Epoch: 2, MSE 0.02514585 (0.02377856), PSNR 15.99533653 (16.23934949), SSIM 0.10503073 (0.11111538)
Finished training epoch 2
Validate: MSE 0.02250111 (0.02380730), PSNR 16.47796059 (16.23853206), SSIM 0.08417158 (0.10861744)
Finished validation.
Starting training epoch 3
Epoch: 3, MSE 0.02398675 (0.02364945), PSNR 16.20028496 (16.26308372), SSIM 0.11610472 (0.11721087)
Finished training epoch 3
Validate: MSE 0.02317484 (0.0

Epoch: 30, MSE 0.02311302 (0.02307502), PSNR 16.36143303 (16.36979072), SSIM 0.15414795 (0.14706006)
Finished training epoch 30
Validate: MSE 0.02274419 (0.02412726), PSNR 16.43129539 (16.18045703), SSIM 0.08376655 (0.10891607)
Finished validation.
Starting training epoch 31
Epoch: 31, MSE 0.02245211 (0.02307363), PSNR 16.48742676 (16.37009028), SSIM 0.15373294 (0.14701323)
Finished training epoch 31
Validate: MSE 0.02354410 (0.02506238), PSNR 16.28117943 (16.01486666), SSIM 0.08224225 (0.10668468)
Finished validation.
Starting training epoch 32
Epoch: 32, MSE 0.02222593 (0.02307425), PSNR 16.53140068 (16.36987498), SSIM 0.16005172 (0.14716723)
Finished training epoch 32
Validate: MSE 0.02255677 (0.02384497), PSNR 16.46722984 (16.23159570), SSIM 0.08896756 (0.11305384)
Finished validation.
Starting training epoch 33
Epoch: 33, MSE 0.02275627 (0.02307747), PSNR 16.42898750 (16.36946130), SSIM 0.15358508 (0.14724829)
Finished training epoch 33
Validate: MSE 0.02309493 (0.02438360), PSNR 

Epoch: 60, MSE 0.02289087 (0.02304769), PSNR 16.40337753 (16.37500277), SSIM 0.16616097 (0.14821929)
Finished training epoch 60
Validate: MSE 0.02241582 (0.02367790), PSNR 16.49445343 (16.26223481), SSIM 0.09270976 (0.11574364)
Finished validation.
Starting training epoch 61
Epoch: 61, MSE 0.02274716 (0.02304445), PSNR 16.43072701 (16.37563492), SSIM 0.15170774 (0.14828142)
Finished training epoch 61
Validate: MSE 0.02241724 (0.02374832), PSNR 16.49417877 (16.24930947), SSIM 0.09249686 (0.11546598)
Finished validation.
Starting training epoch 62
Epoch: 62, MSE 0.02198647 (0.02305331), PSNR 16.57844353 (16.37374437), SSIM 0.15826586 (0.14805472)
Finished training epoch 62
Validate: MSE 0.02258568 (0.02398418), PSNR 16.46166801 (16.20621695), SSIM 0.09172456 (0.11444187)
Finished validation.
Starting training epoch 63
Epoch: 63, MSE 0.02288059 (0.02306368), PSNR 16.40532684 (16.37187748), SSIM 0.13846822 (0.14807468)
Finished training epoch 63
Validate: MSE 0.02238857 (0.02367515), PSNR 

Epoch: 90, MSE 0.02313118 (0.02305143), PSNR 16.35802078 (16.37418053), SSIM 0.14926806 (0.14841684)
Finished training epoch 90
Validate: MSE 0.02240769 (0.02369599), PSNR 16.49602699 (16.25908727), SSIM 0.09094181 (0.11466939)
Finished validation.
Starting training epoch 91
Epoch: 91, MSE 0.02206556 (0.02303868), PSNR 16.56284904 (16.37659773), SSIM 0.15475667 (0.14861198)
Finished training epoch 91
Validate: MSE 0.02240098 (0.02380004), PSNR 16.49732971 (16.24010435), SSIM 0.09063283 (0.11363227)
Finished validation.
Starting training epoch 92
Epoch: 92, MSE 0.02231058 (0.02303065), PSNR 16.51489067 (16.37815748), SSIM 0.14525543 (0.14873846)
Finished training epoch 92
Validate: MSE 0.02249864 (0.02398116), PSNR 16.47843552 (16.20718464), SSIM 0.08654005 (0.11009574)
Finished validation.
Starting training epoch 93
Epoch: 93, MSE 0.02458974 (0.02305609), PSNR 16.09246063 (16.37332672), SSIM 0.13734929 (0.14843594)
Finished training epoch 93
Validate: MSE 0.02238804 (0.02380644), PSNR 

Epoch: 120, MSE 0.02286870 (0.02303496), PSNR 16.40758514 (16.37720037), SSIM 0.15415581 (0.14878128)
Finished training epoch 120
Validate: MSE 0.02263639 (0.02394907), PSNR 16.45192719 (16.21311442), SSIM 0.08793384 (0.11179376)
Finished validation.
Starting training epoch 121
Epoch: 121, MSE 0.02489404 (0.02304600), PSNR 16.03904533 (16.37519488), SSIM 0.13960886 (0.14855184)
Finished training epoch 121
Validate: MSE 0.02261678 (0.02392247), PSNR 16.45569229 (16.21786680), SSIM 0.08835083 (0.11233121)
Finished validation.
Starting training epoch 122
Epoch: 122, MSE 0.02276159 (0.02304230), PSNR 16.42797470 (16.37592310), SSIM 0.15133497 (0.14875336)
Finished training epoch 122
Validate: MSE 0.02243823 (0.02382526), PSNR 16.49011421 (16.23556389), SSIM 0.08825213 (0.11203987)
Finished validation.
Starting training epoch 123
Epoch: 123, MSE 0.02359132 (0.02304582), PSNR 16.27247620 (16.37519375), SSIM 0.14637518 (0.14836115)
Finished training epoch 123
Validate: MSE 0.02250962 (0.02388

<Figure size 432x288 with 0 Axes>

In [20]:
torch.save(model.state_dict(), f'{checkpoints}/last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')

In [21]:
# Validate
save_images = True
with torch.no_grad():
    validate(val_loader, model, criterion, save_images, -1)

Validate: MSE 0.02256579 (0.02403680), PSNR 16.46549416 (16.19734695), SSIM 0.08442003 (0.10843101)
Finished validation.


<Figure size 432x288 with 0 Axes>

In [22]:
# # Show images 
# image_pairs = []

# for i in range(10):
#     image_pairs.append((f'{color_imgs}img-{i}-epoch-{best_epoch}.jpg', f'{gray_imgs}img-{i}-epoch-{best_epoch}.jpg'))
    
# for c, g in image_pairs:
#   color = mpimg.imread(c)
#   gray  = mpimg.imread(g)
#   f, axarr = plt.subplots(1, 2)
#   f.set_size_inches(15, 15)
#   axarr[0].imshow(gray, cmap='gray')
#   axarr[1].imshow(color)
#   axarr[0].axis('off'), axarr[1].axis('off')
#   plt.show()